# Scrape Porn Images 🐙📷

This Notebook scrapes the images to train the `Discriminator` of the **CreateArtPorn Neural Network** from [pornpics.com](https://www.pornpics.com). You can also download pictures manually by:  
1. chosing a category on [www.pornpics.com](https://www.pornpics.com),   
2. scrolling down the page (you need to continue scrolling down as pictures show up "on the fly",   
3. saving website locally (this gives you a among all other files a folder with all images).

Anyhow, as websites html cang change this code might be out of date when you use it. You can test if by running the cell `Code still working?`. If it does not throw any error, chances are high you can use tis notebook:

## Import Libraries

In [1]:
import requests
import pandas as pd
import os
import urllib.request
import time
from pathlib import Path

from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from selenium import webdriver

from urllib.parse import urljoin, urlparse
import shutil #to save locally
import random
import warnings
warnings.filterwarnings('ignore')

## Set Constants

In [2]:
IMAGE_FOLDER = "input/porn-pics" #<- download parent folder fo
CHROMEDRIVER_PATH = "/Applications/chromedriver" #<- get Chromedriver here: https://chromedriver.chromium.org/downloads
URL = "https://www.pornpics.com"

## Code still working?
> Rund this cell to check if the website hasn't changed yet. If everything is fine it should show 2 pictures. **It'll take ca. 1 minute and needs Chromedriver to be installed!**

In [ ]:
# CODE IS WORKING?
from matplotlib import pyplot as plt
driver = webdriver.Chrome(CHROMEDRIVER_PATH)
driver.get(URL)
time.sleep(10)
driver.execute_script("window.scrollTo(1, document.body.scrollHeight);")
time.sleep(5)
soup = bs(driver.page_source, 'html.parser')
driver.close()
main = soup.find_all("div", id="main")
imgs = main[0].find_all("img")
urls = [img["data-src"] for img in imgs]
imgs = random.sample(urls, k=3)
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
i=0
for axe in axes:
    a = plt.imread(imgs[i],format="jpg")
    i+=1
    axe.imshow(a)
plt.axis('off')
plt.show()

## Scrape Categories and Tags
>On the first page are porn genre categories together with links to their main pages.

### Categories

In [4]:
# SCRAPE CATEGORIES
#
# # extract categories and urls
soup = bs(requests.get(URL).content, "html.parser")
elements = soup.find_all("a", {"class": "rel-link"})
for el in elements:
    category = el["href"][1:-1]
    cat_list = [el["href"][1:-1] for el in elements]
    
link_list = ["https://www.pornpics.com/"+cat for cat in cat_list]

# Put tags and links in DataFrame
category_df = pd.DataFrame(columns=["category", "url"], data= {"category": cat_list, "url":link_list})
    #print(category)

# Show results
print("Rows: ", len(category_df))
category_df.head()

Rows:  155


,category,url
0,japanese,https://www.pornpics.com/japanese
1,asian,https://www.pornpics.com/asian
2,european,https://www.pornpics.com/european
3,teen,https://www.pornpics.com/teen
4,pussy,https://www.pornpics.com/pussy


### Tags

In [5]:
url = URL+"/tags/"
soup = bs(requests.get(url).content, "html.parser")

# Scrape tags and links to tag pages
elements = soup.find("div", id= "main-list").find_all("a")
for el in elements:
    tag_list = [el["href"][6:-1] for el in elements if el["href"].startswith("/tags")]
link_list = [URL+tag for tag in tag_list]

# Put tags and links in DataFrame
tags_df = pd.DataFrame(columns=["tags", "url"], data= {"tags": tag_list, "url":link_list})

# Show results
print("Rows: ", len(tags_df))
tags_df.head()

Rows:  1526


,tags,url
0,amateur-anal,https://www.pornpics.comamateur-anal
1,amateur-ass,https://www.pornpics.comamateur-ass
2,amateur-beach,https://www.pornpics.comamateur-beach
3,amateur-bikini,https://www.pornpics.comamateur-bikini
4,amateur-blowjob,https://www.pornpics.comamateur-blowjob


## Scrape Images
>The following cell starts the functions for the Web Crawler. The cell after starts it. In the latter you can decide for which tags and categories the scrawler should download images. The dataframes `tags_categories` and `tags_df` shows you possible values.

In [6]:
# FUNCTIONS TO SCRAPE & DOWNLOAD IMAGES FROM CATEGORY AND/OR TAG

def get_soup(url):
    '''
    Extract html text from a webside where user needs to scroll down to see whole content
    '''
    driver = webdriver.Chrome(CHROMEDRIVER_PATH)
    driver.get(url)
    time.sleep(10)
    this = True
    last = False
    n=0
    while this != last:
        last = this
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        this = driver.page_source
        n+=1
        
    soup = bs(this, 'html.parser')
    
    driver.close()
    
    return soup

def scrape_images(soup, save_path, df, df_failed, category=None, tag=None, seperate_folders=True):
    '''
    Web Crawler to extract all porn images within each category or tag website. 
    '''
    t0=time.time() #<-start timer

    #scrape image
    main = soup.find_all("div", id="main")
    imgs = main[0].find_all("img")
    for img in imgs:
        d = img["alt"] #<-description of image
        u = img["data-src"] #<-url of image
        n = Path(u).name #<-extract file name
        try:
            urllib.request.urlretrieve(u, os.path.join(save_path, n)) #<-download image
            df = df.append({"name":n, "url":u, "description":d, "category":category, "tags":tag}, ignore_index=True)
            
        except: #<-document failed downloads
            df_failed = df_failed.append({"name":n, "url":u, "description":d}, ignore_index=True)

    t1=time.time() #<-stop timer
    print(
        "downloaded {} images out of {} extracted image-urls in {} min and saved in {}".format(
            len(df), len(df)+len(df_failed), int((t1-t0)/60), save_path )
            )
    return df, df_failed

def download_images(categories=None, tags=None):
    t0=time.time() #<-start timer
    #create dataframes for storing
    df = pd.DataFrame(columns=["name", "url", "description", "category", "tags"])
    df_failed = pd.DataFrame(columns=["name", "url", "description"])
    
    # scrape categories
    if categories:
        for cat in categories:
            url = "https://www.pornpics.com/"+cat+"/"         
            try:
                soup = get_soup(url) #<- run get_soup() 💫
                
                # create download folder
                subfolder = "category_"+cat
                dir = os.path.join(IMAGE_FOLDER, subfolder)
                
                if not os.path.exists(dir):
                    os.makedirs(dir)
                try:
                    df, df_failed = scrape_images(soup, dir, df, df_failed, category=cat) #<- run scrape_images() 💫          
                except:
                    print("could not scrape {} from category: {}".format(cat, url))
                    pass
            except:
                print("Could not scrape category: ", cat)

    if tags:
        cat=None
        for tag in tags:
            url = "https://www.pornpics.com/tags/"+tag+"/"
            try:
                soup = get_soup(url) #<- run get_soup() 💫

                # create download folder
                subfolder = "tag_"+tag
                dir = os.path.join(IMAGE_FOLDER, subfolder)
                if not os.path.exists(dir):
                    os.makedirs(dir)
                try:
                    df, df_failed = scrape_images(soup, dir, df, df_failed, tag=tag) #<- run def scrape_images()         
                except:
                    print("could not scrape {} from tag: {}".format(tag, url))
                    pass
            except:
                    print("Could not scrape tag: ", tag)

        t1=time.time() #<-stop timer
        print("Total scraping time: {} min".format( int((t1-t0)/60) ) )
    return df, df_failed

## 👇🏻 🚀🧨 Chose Categories and/or Tags for downloading and start Crawler 🐙

In [7]:
# Chose which categories and tags should be downloaded

# # Select categories and tags by search pattern:
category = category_df[category_df.category.str.contains('blowjob', regex=False)].category.values.tolist()
tags = tags_df[tags_df.tags.str.contains('blowjob', regex=False)].tags.values.tolist()

# or add manually:
categories = ["blowjob"]
tags = ["cum-on-face", "pov-blowjob"]

# RUN CODE (! ca 10min per category, tag)
# ! will open Chrome in the background !
df, df_failed = download_images(categories,tags)

Category blowjob not in <meta_df>!


In [ ]:
print("Rows: ", len(df_failed))
df_failed

Rows:  2


,name,url,description
0,49109844,https://img.strpst.com/us4/previews/1638951709...,Wet__Bunny
1,15739582,https://img.strpst.com/us19/previews/163895185...,madonnaaaa


In [ ]:
print("Rows: ", len(df))
df.head()

Rows:  1000


,name,url,description,category,tags
0,37832869_018_bbed.jpg,https://cdni.pornpics.com/460/1/170/37832869/3...,Petite girl Kimmy Granger takes off sexy linge...,None,pov-blowjob
1,13503756_004_6fdc.jpg,https://cdni.pornpics.com/460/5/250/13503756/1...,Pretty girl Adriana Chechik nearly deepthroats...,None,pov-blowjob
2,98825470_006_4f50.jpg,https://cdni.pornpics.com/460/1/337/98825470/9...,Ebony chick sports braided blonde hair during ...,None,pov-blowjob
3,12710866_009_9c75.jpg,https://cdni.pornpics.com/460/1/123/12710866/1...,Skinny Janice Griffith wraps her big mouth aro...,None,pov-blowjob
4,76914766_001_6587.jpg,https://cdni.pornpics.com/460/5/177/76914766/7...,POV cock sucking and cum swallowing by Summer ...,None,pov-blowjob


In [ ]:
print(df.groupby('category').count().name)
print(df.groupby('tags').count().name)

category
blowjob    1000
Name: name, dtype: int64
tags
cum-on-face    1000
pov-blowjob    1000
Name: name, dtype: int64


## Checking downloaded image files (Optional)

In [ ]:
# ❗️ this function needs PIL-library
def check_images(dir, endings=["jpg","png"]):
    from PIL import Image
    results=[]
    def loop(ending):
        for filename in os.listdir(dir):
            if filename.endswith('.'+ending):
                try:
                    img = Image.open(os.path.join(dir,filename)) # open the image file
                    img.verify() # verify that it is, in fact an image
                except (IOError, SyntaxError) as e:
                    #print('Bad file:', filename) # print out the names of corrupt files
                    results.append(filename)
    for ending in endings:
        loop(ending)

    print("Found {} broken images".format(len(results)))
    return results


for dir in os.listdir(IMAGE_FOLDER):
    dir = os.path.join(IMAGE_FOLDER,dir)
    check_images(dir)

Found 0 broken images
Found 0 broken images
Found 0 broken images


# References
- [Medium: How to Download an Image Using Python](https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c)

---
# Code Friedhof 🏴‍☠️

> NOTHING TO SEE HERE! I just use these section for some code fragments I want to keep - at least for the moment. The `%%script echo skipping` line stops them from executing when pressing `Run All`.

In [ ]:
%%script echo skipping
def download(url, pathname="pics", tid):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, tid)
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

In [ ]:
%%script echo skipping

from PIL import Image

listImages = os.listdir(IMAGE_FOLDER)

for img in listImages:
    if img.startswith("."):
        pass
    else:
        imgPath = os.path.join(imageFolder,img)
                
        try:
            img = Image.open(imgPath)
            exif_data = img._getexif()
        except ValueError as err:
            print(err)
            print("Error on image: ", img)

In [ ]:
%%script echo skipping

# SCRAPE TAGS
# # create DataFrame for storing categories
cat_df = pd.DataFrame(columns=["category", "url", "tags"])

# extract categories and urls
soup = bs(requests.get(URL).content, "html.parser")
tags = soup.find_all("a", {"class": "rel-link"})
for tag in tags:
    category = tag["href"][1:-1]
    src = "https://www.pornpics.com/"+ category +"/"
    cat_df = cat_df.append({"category":category, "url":src}, ignore_index=True)

cat_df.tags=[set()]*len(cat_df) #<-convert column tags to set() to store multiple values
cat_df = cat_df.set_index("category")
# check result
print("Number of categories: ", len(cat_df))
cat_df.head()

Number of categories:  155


,url,tags
category,,
japanese,https://www.pornpics.com/japanese/,{}
asian,https://www.pornpics.com/asian/,{}
european,https://www.pornpics.com/european/,{}
teen,https://www.pornpics.com/teen/,{}
pussy,https://www.pornpics.com/pussy/,{}


In [ ]:
%%script echo skipping
# SCRAPE TAGS
t0=time.time()
not_scraped={}
tag_dict = {}
for index, row in meta_df.iterrows(): 
#for index, row in meta_df.iterrows(): 
    url = row["url"]
    cat = index
    try:
        soup = bs(requests.get(url).content, "html.parser")
        ul = soup.find_all("ul", "clearfix")[0]
        ul = ul.find_all("li")
        for li in ul:
            tag = li.find("a")["href"]
            if "tags" in tag: #some "tags" are categories
                tag=tag[6:-1]
                if cat not in tag_dict:
                    tag_dict[cat] = set()
                tag_dict[cat].add(tag)
                
            else:
                pass
    except:
        not_scraped[tag] = url
        print("could not scrape category {} on {}".format(cat,url))

t1=time.time()
print("Scraping Time: {} min".format(round((t1-t0)/60),1) )

# put scraped tags into meta_df
for k,v in tag_dict.items():
    meta_df.loc[k, "tags"] = v

could not scrape category anal-gape on https://www.pornpics.com/anal-gape/
Scraping Time: 2 min
